In [1]:
datasets = ['CUB', 'Derm7pt', 'RIVAL10']
use_dataset = datasets[2]

In [2]:
import os
import sys

notebook_dir = os.getcwd()
project_root_path = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root_path)

from src.config import CUB_CONFIG, DERM7PT_CONFIG, RIVAL10_CONFIG
from src.config import PROJECT_ROOT
import numpy as np

In [3]:
if use_dataset == 'CUB':
    config_dict = CUB_CONFIG
    DATASET_PATH =  os.path.join(PROJECT_ROOT, 'output', 'CUB')
elif use_dataset == 'Derm7pt':
    config_dict = DERM7PT_CONFIG
    DATASET_PATH =  os.path.join(PROJECT_ROOT, 'output', 'Derm7pt')
else:
    config_dict = RIVAL10_CONFIG
    DATASET_PATH =  os.path.join(PROJECT_ROOT, 'output', 'RIVAL10')

# Load and Transform Data

In [4]:
# INSTANCE-BASED CUB MODEL

# C_train = np.load(os.path.join(PROJECT_ROOT, 'output', 'CUB', 'C_train_instance.npy'))
# C_hat_train = np.load(os.path.join(PROJECT_ROOT, 'output', 'CUB', 'C_hat_sigmoid_train_instance.npy'))
# one_hot_Y_train = np.load(os.path.join(PROJECT_ROOT, 'output', 'CUB', 'Y_train_instance.npy'))

# C_test = np.load(os.path.join(PROJECT_ROOT, 'output', 'CUB', 'C_test_instance.npy'))
# C_hat_test = np.load(os.path.join(PROJECT_ROOT, 'output', 'CUB', 'C_hat_sigmoid_test_instance.npy'))
# one_hot_Y_test = np.load(os.path.join(PROJECT_ROOT, 'output', 'CUB', 'Y_test_instance.npy'))

In [5]:
C_hat_train = np.load(os.path.join(DATASET_PATH, 'C_hat_sigmoid_train.npy'))
one_hot_Y_train = np.load(os.path.join(DATASET_PATH, 'Y_train.npy'))

C_hat_test = np.load(os.path.join(DATASET_PATH, 'C_hat_sigmoid_test.npy'))
one_hot_Y_test = np.load(os.path.join(DATASET_PATH, 'Y_test.npy'))

if use_dataset == 'Derm7pt':
    C_hat_val = np.load(os.path.join(DATASET_PATH, 'C_hat_sigmoid_val.npy'))
    one_hot_Y_val = np.load(os.path.join(DATASET_PATH, 'Y_val.npy'))

class_level_concepts = np.load(os.path.join(DATASET_PATH, 'class_level_concepts.npy'))

In [6]:
Y_train = np.argmax(one_hot_Y_train, axis=1)
Y_test = np.argmax(one_hot_Y_test, axis=1)

In [7]:
# unique, counts = np.unique(Y_train, return_counts=True)
# for label, count in zip(unique, counts):
#     print(f"Label {label}: {count} instances")

In [8]:
C_train = []
for y in Y_train:
    C_train.append(class_level_concepts[y])

C_train = np.array(C_train)

In [9]:
# from sklearn.utils import shuffle


# C_hat_train, C_train, one_hot_Y_train, Y_train = shuffle(C_hat_train, C_train, one_hot_Y_train, Y_train, random_state=42)

In [10]:
# C_hat_train[C_hat_train < 0.1] = 0
# C_hat_test[C_hat_test < 0.1] = 0

In [11]:
# unique, counts = np.unique(Y_test, return_counts=True)
# for label, count in zip(unique, counts):
#     print(f"Label {label}: {count} instances")

# Classic Models

## Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(C_hat_train, Y_train)
print(f"Logistic Regression Test accuracy: {model.score(C_hat_test, Y_test)}")

Logistic Regression Test accuracy: 0.9916761256148316


In [13]:
np.unique(model.predict(C_hat_test))

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

## k-NN

In [14]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(C_hat_train, Y_train)
print(f"k-NN Test accuracy: {model.score(C_hat_test, Y_test)}")

k-NN Test accuracy: 0.9899735149451381


In [15]:
np.unique(model.predict(C_hat_test))

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

## Decision Tree

In [16]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(C_hat_train, Y_train)
print(f"Decision Tree Test accuracy: {model.score(C_hat_test, Y_test)}")

Decision Tree Test accuracy: 0.9858115777525539


In [17]:
np.unique(model.predict(C_hat_test))

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

## MLP

In [18]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(512,256, 128), max_iter=1000)
mlp.fit(C_hat_train, Y_train)
print(f"MLP Test accuracy: {mlp.score(C_hat_test, Y_test)}")

MLP Test accuracy: 0.9892167990919409


In [19]:
np.unique(model.predict(C_hat_test))

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Accuracy Using Class-Level Concepts

In [20]:
# Function to find the closest concept vector and predict the label
def predict_nearest_concept(instance, reference_concepts, reference_labels):
    distances = np.sqrt(np.sum((reference_concepts - instance)**2, axis=1))
    min_idx = np.argmin(distances)
    return reference_labels[min_idx]

# Use C_train as reference concepts and evaluate on C_hat_test
correct_predictions = 0
total_predictions = len(C_hat_test)

for i, test_instance in enumerate(C_hat_test):
    predicted_label = predict_nearest_concept(test_instance, C_train, Y_train)
    true_label = Y_test[i]

    if predicted_label == true_label:
        correct_predictions += 1

# Calculate and print accuracy
accuracy = correct_predictions / total_predictions
print(f"\nOverall accuracy using concept-based nearest neighbor: {accuracy:.4f}")


Overall accuracy using concept-based nearest neighbor: 0.9896


# Prototype-Based Model


In [21]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


## Create Dataloaders

In [23]:
val_split_ratio = 0.2
random_seed = 42

if use_dataset == 'Derm7pt':
    X_train = torch.tensor(C_hat_train, dtype=torch.float32)
    Y_train = torch.tensor(one_hot_Y_train, dtype=torch.float32)
    X_val = torch.tensor(C_hat_val, dtype=torch.float32)
    Y_val = torch.tensor(one_hot_Y_val, dtype=torch.float32)
else:
    C_hat_train, C_hat_val, Y_train_np, Y_val_np = train_test_split(C_hat_train, one_hot_Y_train, test_size=val_split_ratio, random_state=random_seed)
    X_train = torch.tensor(C_hat_train, dtype=torch.float32)
    Y_train = torch.tensor(Y_train_np, dtype=torch.float32)
    X_val = torch.tensor(C_hat_val, dtype=torch.float32)
    Y_val = torch.tensor(Y_val_np, dtype=torch.float32)

X_test = torch.tensor(C_hat_test, dtype=torch.float32, device=device)
Y_test = torch.tensor(one_hot_Y_test, dtype=torch.float32, device=device)

# DATALOADERS
batch_size = 64
train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(X_val, Y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TensorDataset(X_test, Y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Learn Prototypes

In [24]:
from src.models import PrototypeClassifier

num_concepts = config_dict['N_TRIMMED_CONCEPTS']
num_classes = config_dict['N_CLASSES']

model = PrototypeClassifier(num_concepts, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
lambda_binary = 0.01
lambda_L1 = 0.001

In [25]:
# train and test
from src.training import train_epoch, val_epoch

num_epochs = 50

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_epoch(model, train_loader, optimizer, lambda_binary, lambda_L1, device=device)
    val_accuracy = val_epoch(model, val_loader, device)
    if((epoch+1)%10==0):
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")

Epoch [10/50]
Train Loss: -6.5228, Train Accuracy: 99.16%, Validation Accuracy: 99.15%
Epoch [20/50]
Train Loss: -6.5574, Train Accuracy: 99.16%, Validation Accuracy: 99.15%
Epoch [30/50]
Train Loss: -6.5621, Train Accuracy: 99.16%, Validation Accuracy: 99.17%
Epoch [40/50]
Train Loss: -6.5633, Train Accuracy: 99.16%, Validation Accuracy: 99.17%
Epoch [50/50]
Train Loss: -6.5633, Train Accuracy: 99.15%, Validation Accuracy: 99.17%


In [26]:
real_labels = Y_test.argmax(dim=1)
predictions = model.predict(X_test)
(predictions == real_labels).sum().item()/len(predictions)

0.9895951570185395

In [27]:
np.unique(predictions.cpu().numpy())

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [28]:
from sklearn.metrics import classification_report

y_true = real_labels.cpu().numpy()
y_pred = predictions.cpu().numpy()

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       533
           1       0.99      0.99      0.99       534
           2       1.00      0.99      0.99       532
           3       0.99      0.99      0.99       530
           4       0.99      1.00      0.99       531
           5       0.99      0.99      0.99       533
           6       0.99      1.00      1.00       530
           7       0.99      0.98      0.98       529
           8       0.99      1.00      0.99       532
           9       0.97      0.98      0.98       502

    accuracy                           0.99      5286
   macro avg       0.99      0.99      0.99      5286
weighted avg       0.99      0.99      0.99      5286



In [29]:
close_to_zero = (torch.sum((model.get_sigmoid_prototypes() < 0.1) | (model.get_sigmoid_prototypes() > 0.9)) / (200*112)).cpu().numpy()
print(f"{close_to_zero*100}% of the values are close to 0 or 1")

0.8035714626312256% of the values are close to 0 or 1


# Class-level vs Learned

In [30]:
print(class_level_concepts)

# Overall sparsity (fraction of zeros)
overall_sparsity = np.mean(class_level_concepts == 0)
print(f"Overall sparsity: {overall_sparsity:.4f}")

# Sparsity per row (fraction of zeros in each row)
row_sparsity = np.mean(class_level_concepts == 0, axis=1)
print("Sparsity per row:", row_sparsity)

[[0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0]]
Overall sparsity: 0.7500
Sparsity per row: [0.83333333 0.83333333 0.77777778 0.72222222 0.77777778 0.66666667
 0.88888889 0.61111111 0.72222222 0.66666667]


In [31]:
Prototypes = model.get_sigmoid_prototypes()
Prototypes = Prototypes.cpu().detach().numpy()
Prototypes[Prototypes>=0.5] = 1
Prototypes[Prototypes<0.5]= 0
print(Prototypes)

# Overall sparsity (fraction of zeros)
overall_sparsity = np.mean(Prototypes == 0)
print(f"Overall sparsity: {overall_sparsity:.4f}")

# Sparsity per row (fraction of zeros in each row)
row_sparsity = np.mean(Prototypes == 0, axis=1)
print("Sparsity per row:", row_sparsity)

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0.]]
Overall sparsity: 0.7500
Sparsity per row: [0.83333333 0.83333333 0.77777778 0.72222222 0.77777778 0.66666667
 0.88888889 0.61111111 0.72222222 0.66666667]


# MY OLD CODE

In [32]:
# # --- Plotting ---
# from matplotlib import pyplot as plt

# plt.figure(figsize=(10, 5))
# epochs_range = range(1, epochs + 1)
# plt.plot(epochs_range, train_losses, label='Training Loss', marker='o', linestyle='-')
# plt.plot(epochs_range, val_losses, label='Validation Loss', marker='x', linestyle='--')
# plt.title('Training and Validation Loss Over Epochs')
# plt.xlabel('Epoch')
# plt.ylabel('Average Loss')
# plt.legend()
# plt.grid(True)
# plt.show()

# # Optional: Plot validation accuracy as well
# plt.figure(figsize=(10, 5))
# plt.plot(epochs_range, val_accuracies, label='Validation Accuracy', marker='s', linestyle='-', color='green')
# plt.title('Validation Accuracy Over Epochs')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy (%)')
# plt.legend()
# plt.grid(True)
# plt.show()

In [33]:
# prototypes = []
# for y in Y_train:
#     prototypes.append(final_binary_prototypes[y])

# prototypes = np.array(prototypes)

In [34]:
# # Function to find the closest concept vector and predict the label
# def predict_nearest_concept(instance, reference_concepts, reference_labels):
#     distances = np.sum(np.abs(reference_concepts - instance), axis=1)
#     min_idx = np.argmin(distances)
#     return reference_labels[min_idx]

# # Use prototypes as reference concepts and evaluate on C_hat_test
# correct_predictions = 0
# total_predictions = len(C_hat_test)

# for i, test_instance in enumerate(C_hat_test):
#     predicted_label = predict_nearest_concept(test_instance, prototypes, Y_train)
#     true_label = Y_test[i]

#     if predicted_label == true_label:
#         correct_predictions += 1

# # Calculate and print accuracy
# accuracy = correct_predictions / total_predictions
# print(f"\nOverall accuracy using prototype-based nearest neighbor: {accuracy:.4f}")